In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/google/txgemma-9b-chat

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/txgemma-9b-chat)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/txgemma-9b-chat")
model = AutoModelForCausalLM.from_pretrained("google/txgemma-9b-chat")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

In [ ]:
# 경고 메시지(ERROR: pip's dependency resolver...)가 떠도 무시하고 진행하세요.
# 관련 라이브러리 설치
!pip install -q -U langchain langchain-community langchain-huggingface sentence-transformers chromadb accelerate bitsandbytes

In [ ]:
!pip install -U bitsandbytes

In [ ]:
# rank_bm25(키워드 검색용) 설치
!pip install -q rank_bm25

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from huggingface_hub import login

# ---------------------------------------------------------------
class MedicalSmartRAG:
    def __init__(
        self,
        model_id: str = "google/txgemma-9b-chat",
        embedding_model: str = "jhgan/ko-sroberta-multitask"
    ):
        print(f"🚀 [{model_id}] RAG 파이프라인 초기화 중...")

        # 1. 4-bit 양자화 설정
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

        # 2. 모델 및 토크나이저 로드
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto", # GPU 자동 할당
            trust_remote_code=True
        )

        # 3. 한국어 임베딩 모델 로드
        print(f"📊 임베딩 모델 로드: {embedding_model}")
        self.embed_model = SentenceTransformer(embedding_model)

        # 4. 벡터 DB (Chroma) 초기화
        self.chroma_client = chromadb.Client(Settings(allow_reset=True))
        try:
            self.chroma_client.delete_collection("medical_kb")
        except:
            pass
        self.collection = self.chroma_client.create_collection("medical_kb")

        print("✅ 시스템 준비 완료!")

    def ingest_documents(self, documents: list):
        """
        문서 리스트를 벡터 DB에 저장
        documents = [{"text": "...", "source": "..."}, ...]
        """
        print(f"📚 문서 {len(documents)}개 처리 중...")
        texts = [doc['text'] for doc in documents]
        metadatas = [{"source": doc.get('source', 'unknown')} for doc in documents]
        ids = [f"doc_{i}" for i in range(len(texts))]

        # 임베딩 생성
        embeddings = self.embed_model.encode(texts, convert_to_numpy=True)

        # DB 저장
        self.collection.add(
            documents=texts,
            embeddings=embeddings.tolist(),
            metadatas=metadatas,
            ids=ids
        )
        print("💾 저장 완료.")

    def generate_response(self, query: str):
        # 1. 검색 (Retrieval)
        query_vec = self.embed_model.encode(query).tolist()
        results = self.collection.query(query_embeddings=[query_vec], n_results=3)

        retrieved_docs = results['documents'][0]
        sources = [meta['source'] for meta in results['metadatas'][0]]

        # 문맥 조합
        context_str = "\n\n".join([f"- 내용: {text}" for text in retrieved_docs])

        # 2. 프롬프트 구성 (Gemma Chat Template 준수)
        # TxGemma는 의료 특화이므로 전문적인 페르소나를 부여합니다.
        prompt = f"""<start_of_turn>user
당신은 한국의 의료 전문가 AI입니다. 아래 제공된 [의료 지침]을 바탕으로 질문에 대해 전문적이고 명확하게 한국어로 답변하세요.


주의사항:
1. 수치가 '권고(Target)'하는 것인지, '피해야 할(Warning)' 것인지 문맥을 정확히 파악하세요.
2. 'J-모양 관련성'이나 '사망률 증가'와 관련된 수치는 목표 수치가 아닙니다.
3. 지침에 없는 내용은 모른다고 답하세요.

[의료 지침]
{context_str}

질문: {query}<end_of_turn>
<start_of_turn>model
"""

        # 3. 답변 생성 (Generation)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=512,       # 답변 길이
                temperature=0.2,          # 사실 기반이므로 낮게 설정
                repetition_penalty=1.1,   # 반복 방지
                do_sample=True
            )

        # 4. 결과 디코딩
        answer = self.tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

        return {
            "query": query,
            "answer": answer.strip(),
            "sources": list(set(sources)) # 중복 제거
        }

In [ ]:
!pip install -q langchain-text-splitters

In [ ]:
#pdf 표 인식을 위한 라이브러ㅣ
!pip install pdfplumber

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# 1. 시스템 초기화
rag_bot = MedicalSmartRAG()

import pdfplumber
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def table_to_markdown(table):
    """
    pdfplumber로 추출한 표(List[List[str]])를 Markdown 형식의 문자열로 변환
    """
    if not table or len(table) < 2:
        return ""

    # None 값을 빈 문자열로 변환
    table = [['' if cell is None else cell for cell in row] for row in table]

    try:
        # 헤더 생성
        markdown = "| " + " | ".join(map(str, table[0])) + " |\n"
        markdown += "| " + " | ".join(["---"] * len(table[0])) + " |\n"

        # 데이터 행 생성
        for row in table[1:]:
            markdown += "| " + " | ".join(map(str, row)) + " |\n"
        return markdown + "\n"
    except Exception as e:
        return "" # 변환 실패 시 빈 문자열 반환

def process_pdf_with_tables(file_path):
    print(f"📄 PDF 로딩 및 표 처리 중: {file_path}")
    docs = []

    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            # 1. 텍스트 추출
            text = page.extract_text() or ""

            # 2. 표 추출 및 마크다운 변환
            tables = page.extract_tables()
            table_text = ""
            if tables:
                for table in tables:
                    table_text += table_to_markdown(table)

            # 3. 텍스트와 표 내용을 합침 (표가 텍스트 아래에 오도록)
            # 팁: 표가 중요하다면 table_text를 text보다 앞에 둘 수도 있습니다.
            combined_content = f"{text}\n\n[표 데이터]\n{table_text}"

            # 4. LangChain Document 객체 생성
            docs.append(Document(
                page_content=combined_content,
                metadata={"source": f"{file_path} (Page {i + 1})", "page": i + 1}
            ))

    # 5. 텍스트 분할 (Chunking)
    # Markdown 형식이 깨지지 않도록 청크 사이즈를 조금 더 넉넉하게 잡는 것이 좋습니다.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,       # 표가 포함되므로 크기를 늘림 (500 -> 800)
        chunk_overlap=350,
        separators=["\n\n", "\n", "|", ".", " "] # 표의 구분자('|')를 separator에 추가
    )

    splits = text_splitter.split_documents(docs)

    # 6. 데이터 포맷 변환 (MedicalSmartRAG 호환용)
    processed_docs = []
    for doc in splits:
        processed_docs.append({
            "text": doc.page_content,
            "source": doc.metadata['source']
        })

    print(f"✅ 총 {len(docs)}페이지를 {len(processed_docs)}개의 청크(표 포함)로 분할 완료.")
    return processed_docs

# --- 실행부 ---

# 업로드한 PDF 파일 경로
pdf_path = "/content/drive/MyDrive/[대한의학회] 당뇨병 임상진료지침.pdf"

# 새로운 함수로 실행
real_guidelines = process_pdf_with_tables(pdf_path)

# 기존 RAG 봇에 주입
rag_bot.ingest_documents(real_guidelines)

In [ ]:
# 4. 질문하기
q1 = input("질문: ")
result = rag_bot.generate_response(q1)
print(f"답변:\n{result['answer']}")
print(f"출처: {result['sources']}")